In [5]:
using JuMP
using Cbc
using DataFrames
using Gurobi
using CSV

In [8]:
# m = Model(solver = CbcSolver())
m = Model(solver=GurobiSolver(Presolve=0))

data = CSV.read("supermarket_data.csv")
simple_dist = CSV.read("distmatrix_simple.csv")

v = data[:,2]

simple_dist = simple_dist[:,2:103]
c = simple_dist
    
@variables m begin
    y[1:102], Bin
end

@variables m begin
    x[1:102,1:102], Bin
end

@variables m begin
    z[1:102]
end

@variables m begin
    t[1:102,1:102]
end

@constraint(m, y[1] == 1)
@constraint(m, y[102] == 1)
@constraint(m, z[1] == 0)
@constraint(m, sum(y[i] for i in 2:101) <= 15)

for i in 1:101
    @constraint(m, sum(x[i,j] for j in 1:102) == y[i])
end

for j in 2:102
    @constraint(m, sum(x[i,j] for i in 1:102) == y[j])
end

for j in 2:102
    @constraint(m, sum(t[i,j] for i in 1:102) == z[j])
end

for j in 1:101
    @constraint(m, sum(t[j,k] for k in 1:102) == (z[j] + sum(c[j,k]x[j,k] for k in 1:102)))
end

for j in 1:102
    for k in 1:102
        @constraint(m, t[j,k] >= 0)
        @constraint(m, t[j,k] <= 90x[j,k])
    end
end

@objective(m, Max, sum(v[i]*y[i] for i in 1:102))

print(m)
solve(m)



Max 6.99 y[2] + 10.99 y[3] + 3.19 y[4] + 3.99 y[5] + 3.29 y[6] + 3.69 y[7] + 5.49 y[8] + 2.69 y[9] + 6.99 y[10] + 4.99 y[11] + 2.49 y[12] + 4.99 y[13] + 2.29 y[14] + 1.69 y[15] + 3.99 y[16] + 1.99 y[17] + 2.69 y[18] + 2 y[19] + 2.49 y[20] + 3.49 y[21] + 2.99 y[22] + 3.99 y[23] + 3.49 y[24] + 2.49 y[25] + 3.49 y[26] + 2.49 y[27] + 1.99 y[28] + 2.99 y[29] + 1.99 y[30] + 3.99 y[31] + 2.99 y[32] + 5.49 y[33] + 25.99 y[34] + 0.88 y[35] + 2.99 y[36] + 6.29 y[37] + 4.99 y[38] + 3.99 y[39] + 5.29 y[40] + 8.99 y[41] + 7.99 y[42] + 9.99 y[43] + 6.49 y[44] + 3.29 y[45] + 5.69 y[46] + 3.49 y[47] + 8.99 y[48] + 2.99 y[49] + 12.99 y[50] + 4.99 y[51] + 13.99 y[52] + 3.99 y[53] + 7.99 y[54] + 3.99 y[55] + 6.99 y[56] + 3.99 y[57] + 3.49 y[58] + 3.49 y[59] + 2.49 y[60] + 6.99 y[61] + 7.99 y[62] + 3.99 y[63] + 5.99 y[64] + 5.49 y[65] + 5.99 y[66] + 1.69 y[67] + 3.99 y[68] + 3.99 y[69] + 6.99 y[70] + 4.99 y[71] + 3.79 y[72] + 2.99 y[73] + 3.99 y[74] + 4.49 y[75] + 2.99 y[76] + 3.99 y[77] + 3.99 y[78] + 3.

 x[1,16] + x[2,16] + x[3,16] + x[4,16] + x[5,16] + x[6,16] + x[7,16] + x[8,16] + x[9,16] + x[10,16] + x[11,16] + x[12,16] + x[13,16] + x[14,16] + x[15,16] + x[16,16] + x[17,16] + x[18,16] + x[19,16] + x[20,16] + x[21,16] + x[22,16] + x[23,16] + x[24,16] + x[25,16] + x[26,16] + x[27,16] + x[28,16] + x[29,16] + x[30,16] + x[31,16] + x[32,16] + x[33,16] + x[34,16] + x[35,16] + x[36,16] + x[37,16] + x[38,16] + x[39,16] + x[40,16] + x[41,16] + x[42,16] + x[43,16] + x[44,16] + x[45,16] + x[46,16] + x[47,16] + x[48,16] + x[49,16] + x[50,16] + x[51,16] + x[52,16] + x[53,16] + x[54,16] + x[55,16] + x[56,16] + x[57,16] + x[58,16] + x[59,16] + x[60,16] + x[61,16] + x[62,16] + x[63,16] + x[64,16] + x[65,16] + x[66,16] + x[67,16] + x[68,16] + x[69,16] + x[70,16] + x[71,16] + x[72,16] + x[73,16] + x[74,16] + x[75,16] + x[76,16] + x[77,16] + x[78,16] + x[79,16] + x[80,16] + x[81,16] + x[82,16] + x[83,16] + x[84,16] + x[85,16] + x[86,16] + x[87,16] + x[88,16] + x[89,16] + x[90,16] + x[91,16] + x[92,16

 x[1,74] + x[2,74] + x[3,74] + x[4,74] + x[5,74] + x[6,74] + x[7,74] + x[8,74] + x[9,74] + x[10,74] + x[11,74] + x[12,74] + x[13,74] + x[14,74] + x[15,74] + x[16,74] + x[17,74] + x[18,74] + x[19,74] + x[20,74] + x[21,74] + x[22,74] + x[23,74] + x[24,74] + x[25,74] + x[26,74] + x[27,74] + x[28,74] + x[29,74] + x[30,74] + x[31,74] + x[32,74] + x[33,74] + x[34,74] + x[35,74] + x[36,74] + x[37,74] + x[38,74] + x[39,74] + x[40,74] + x[41,74] + x[42,74] + x[43,74] + x[44,74] + x[45,74] + x[46,74] + x[47,74] + x[48,74] + x[49,74] + x[50,74] + x[51,74] + x[52,74] + x[53,74] + x[54,74] + x[55,74] + x[56,74] + x[57,74] + x[58,74] + x[59,74] + x[60,74] + x[61,74] + x[62,74] + x[63,74] + x[64,74] + x[65,74] + x[66,74] + x[67,74] + x[68,74] + x[69,74] + x[70,74] + x[71,74] + x[72,74] + x[73,74] + x[74,74] + x[75,74] + x[76,74] + x[77,74] + x[78,74] + x[79,74] + x[80,74] + x[81,74] + x[82,74] + x[83,74] + x[84,74] + x[85,74] + x[86,74] + x[87,74] + x[88,74] + x[89,74] + x[90,74] + x[91,74] + x[92,74

 t[1,32] + t[2,32] + t[3,32] + t[4,32] + t[5,32] + t[6,32] + t[7,32] + t[8,32] + t[9,32] + t[10,32] + t[11,32] + t[12,32] + t[13,32] + t[14,32] + t[15,32] + t[16,32] + t[17,32] + t[18,32] + t[19,32] + t[20,32] + t[21,32] + t[22,32] + t[23,32] + t[24,32] + t[25,32] + t[26,32] + t[27,32] + t[28,32] + t[29,32] + t[30,32] + t[31,32] + t[32,32] + t[33,32] + t[34,32] + t[35,32] + t[36,32] + t[37,32] + t[38,32] + t[39,32] + t[40,32] + t[41,32] + t[42,32] + t[43,32] + t[44,32] + t[45,32] + t[46,32] + t[47,32] + t[48,32] + t[49,32] + t[50,32] + t[51,32] + t[52,32] + t[53,32] + t[54,32] + t[55,32] + t[56,32] + t[57,32] + t[58,32] + t[59,32] + t[60,32] + t[61,32] + t[62,32] + t[63,32] + t[64,32] + t[65,32] + t[66,32] + t[67,32] + t[68,32] + t[69,32] + t[70,32] + t[71,32] + t[72,32] + t[73,32] + t[74,32] + t[75,32] + t[76,32] + t[77,32] + t[78,32] + t[79,32] + t[80,32] + t[81,32] + t[82,32] + t[83,32] + t[84,32] + t[85,32] + t[86,32] + t[87,32] + t[88,32] + t[89,32] + t[90,32] + t[91,32] + t[92,32

 t[1,90] + t[2,90] + t[3,90] + t[4,90] + t[5,90] + t[6,90] + t[7,90] + t[8,90] + t[9,90] + t[10,90] + t[11,90] + t[12,90] + t[13,90] + t[14,90] + t[15,90] + t[16,90] + t[17,90] + t[18,90] + t[19,90] + t[20,90] + t[21,90] + t[22,90] + t[23,90] + t[24,90] + t[25,90] + t[26,90] + t[27,90] + t[28,90] + t[29,90] + t[30,90] + t[31,90] + t[32,90] + t[33,90] + t[34,90] + t[35,90] + t[36,90] + t[37,90] + t[38,90] + t[39,90] + t[40,90] + t[41,90] + t[42,90] + t[43,90] + t[44,90] + t[45,90] + t[46,90] + t[47,90] + t[48,90] + t[49,90] + t[50,90] + t[51,90] + t[52,90] + t[53,90] + t[54,90] + t[55,90] + t[56,90] + t[57,90] + t[58,90] + t[59,90] + t[60,90] + t[61,90] + t[62,90] + t[63,90] + t[64,90] + t[65,90] + t[66,90] + t[67,90] + t[68,90] + t[69,90] + t[70,90] + t[71,90] + t[72,90] + t[73,90] + t[74,90] + t[75,90] + t[76,90] + t[77,90] + t[78,90] + t[79,90] + t[80,90] + t[81,90] + t[82,90] + t[83,90] + t[84,90] + t[85,90] + t[86,90] + t[87,90] + t[88,90] + t[89,90] + t[90,90] + t[91,90] + t[92,90

 t[21,1] + t[21,2] + t[21,3] + t[21,4] + t[21,5] + t[21,6] + t[21,7] + t[21,8] + t[21,9] + t[21,10] + t[21,11] + t[21,12] + t[21,13] + t[21,14] + t[21,15] + t[21,16] + t[21,17] + t[21,18] + t[21,19] + t[21,20] + t[21,21] + t[21,22] + t[21,23] + t[21,24] + t[21,25] + t[21,26] + t[21,27] + t[21,28] + t[21,29] + t[21,30] + t[21,31] + t[21,32] + t[21,33] + t[21,34] + t[21,35] + t[21,36] + t[21,37] + t[21,38] + t[21,39] + t[21,40] + t[21,41] + t[21,42] + t[21,43] + t[21,44] + t[21,45] + t[21,46] + t[21,47] + t[21,48] + t[21,49] + t[21,50] + t[21,51] + t[21,52] + t[21,53] + t[21,54] + t[21,55] + t[21,56] + t[21,57] + t[21,58] + t[21,59] + t[21,60] + t[21,61] + t[21,62] + t[21,63] + t[21,64] + t[21,65] + t[21,66] + t[21,67] + t[21,68] + t[21,69] + t[21,70] + t[21,71] + t[21,72] + t[21,73] + t[21,74] + t[21,75] + t[21,76] + t[21,77] + t[21,78] + t[21,79] + t[21,80] + t[21,81] + t[21,82] + t[21,83] + t[21,84] + t[21,85] + t[21,86] + t[21,87] + t[21,88] + t[21,89] + t[21,90] + t[21,91] + t[21,92

Excessive output truncated after 524288 bytes.

:Optimal